In [2]:
import os
from Bio import SeqIO

In [3]:
def ucitaj_povrsinske_proteine(putanja_do_fajla):
    povrsinski_proteini_nazivi = {}
    povrsinski_proteini_rnk = {}
    with open(putanja_do_fajla, "r") as fajl:
        for red in SeqIO.parse(fajl, "fasta"):
            if any(kljucna_rec in red.description for kljucna_rec in ['spike', 'surface']):
                povrsinski_proteini_nazivi[red.name] = red.description.split(" | ")[1]
                povrsinski_proteini_rnk[red.name] = red.seq

    return povrsinski_proteini_nazivi, povrsinski_proteini_rnk

In [11]:
putanja_do_fajla = "podaci/proteini/kodoni.fasta"
povrsinski_proteini_nazivi, povrsinski_proteini_rnk = ucitaj_povrsinske_proteine(putanja_do_fajla)

In [12]:
for virus, rnk in povrsinski_proteini_rnk.items():
    print(virus, len(rnk))

mers 4062
bcov 4092
sars1 3768
human229e 3522
humanoc43 4062


In [13]:
# Prevodjenje kodona u p-adicne brojeve
def kodon_u_p_adicni_broj(kodon):
    kodon_u_broj = {
        'CCC': 111, 'CCU': 113, 'CCA': 112, 'CCG': 114,
        'CAC': 121, 'CAU': 123, 'CAA': 122, 'CAG': 124,
        'CUC': 131, 'CUU': 133, 'CUA': 132, 'CUG': 134,
        'CGC': 141, 'CGU': 143, 'CGA': 142, 'CGG': 144,
        'ACC': 211, 'ACU': 213, 'ACA': 212, 'ACG': 214,
        'AAC': 221, 'AAU': 223, 'AAA': 222, 'AAG': 224,
        'AUC': 231, 'AUU': 233, 'AUA': 232, 'AUG': 234,
        'AGC': 241, 'AGU': 243, 'AGA': 242, 'AGG': 244,
        'UCC': 311, 'UCU': 313, 'UCA': 312, 'UCG': 314,
        'UAC': 321, 'UAU': 323, 'UAA': 322, 'UAG': 324,
        'UUC': 331, 'UUU': 333, 'UUA': 332, 'UUG': 334,
        'UGC': 341, 'UGU': 343, 'UGA': 342, 'UGG': 344,
        'GCC': 411, 'GCU': 413, 'GCA': 412, 'GCG': 414,
        'GAC': 421, 'GAU': 423, 'GAA': 422, 'GAG': 424,
        'GUC': 431, 'GUU': 433, 'GUA': 432, 'GUG': 434,
        'GGC': 441, 'GGU': 443, 'GGA': 442, 'GGG': 444
    }

    return kodon_u_broj[kodon]
        
# Prevodjenje RNK u niz p-adicnih brojeva
def rnk_sekvenca_u_p_adicne_brojeve(rnk_sekvenca):
    brojevi = []
    for i in range(0, len(rnk_sekvenca), 3):
        kodon = rnk_sekvenca[i:i+3]
        brojevi.append(kodon_u_p_adicni_broj(kodon))  
    return brojevi

In [14]:
def p_adicno_rastojanje_kodona(x, y, p):
    x, y = str(x), str(y)
    
    if x[0] != y[0]:
        # print(x, y, 1)
        return 1
    elif x[1] != y[1]:
        # print(x, y, 1/p)
        return 1/p
    elif x[2] != y[2]:
        # print(x, y, 1/p**2)
        return 1/(p**2)

    # print(x, y, 0)   
    return 0
    
def p_adicno_rastojanje(p_rnk1, p_rnk2, p=5):
    p_rastojanje = 0

    for (x, y) in zip(p_rnk1, p_rnk2):
        p_rastojanje += p_adicno_rastojanje_kodona(x, y, p)
           
    return p_rastojanje

In [26]:
def rnk_u_kodone(rnk):
    kodoni = [rnk[i:i+3] for i in range(0, len(rnk), 3)]
    return kodoni

In [25]:
import copy 
import numpy as np
from Bio.Seq import Seq

# s obzirom da duzine rnk koje kodiraju povrsinske proteine nisu jednake duzine, moramo da im izjednacimo duzine

# 1. pristup: kracu sekvencu dopunimo do duzine duze sekvence najucestalijim kodonom
def izjednacavanje_sekvenci_kodona(sekv1, sekv2):
    sekv1, sekv2 = rnk_u_kodone(sekv1), rnk_u_kodone(sekv2)
    
    duza_sekvenca = sekv1 if len(sekv1) > len(sekv2) else sekv2
    kraca_sekvenca = sekv1 if len(duza_sekvenca) == len(sekv2) else sekv2
    ucestalost_kodona = {}
    
    for kodon in kraca_sekvenca:
        if kodon in ucestalost_kodona:
            ucestalost_kodona[kodon] += 1
        else:
            ucestalost_kodona[kodon] = 1
        
    najcesci_kodon = max(ucestalost_kodona, key=ucestalost_kodona.get)
    broj_pojavljivanja = ucestalost_kodona[najcesci_kodon]

    # Dopunimo kraću sekvencu sa najčešćim kodonom
    broj_kodona_dodati = len(duza_sekvenca) - len(kraca_sekvenca)
    dopunjeni_kodoni = [najcesci_kodon] * broj_kodona_dodati

    # da se ne bi iz nekog razloga direktno menjale vrednosti, pravimo kopiju 
    dopunjena_sekvenca = kraca_sekvenca.copy()
    dopunjena_sekvenca.extend(dopunjeni_kodoni)

    dopunjena_sekvenca = ''.join(str(seq) for seq in dopunjena_sekvenca)
    duza_sekvenca = ''.join(str(seq) for seq in duza_sekvenca)
    # print(dopunjena_sekvenca)

    
    return dopunjena_sekvenca, duza_sekvenca


# 2. pristup: kracu sekvencu dopunimo do duzine duze sekvence najucestalijim nukleotidom
def izjednacavanje_sekvenci_nukleotida(sek1, sek2):
    duza_sekvenca = sek1 if len(sek1) > len(sek2) else sek2
    kraca_sekvenca = sek1 if len(duza_sekvenca) == len(sek2) else sek2
    ucestalost_nukleotida = {}
    
    for nukleotid in kraca_sekvenca:
        if nukleotid in ucestalost_nukleotida:
            ucestalost_nukleotida[nukleotid] += 1
        else:
            ucestalost_nukleotida[nukleotid] = 1
        
    najcesci_nukleotid = max(ucestalost_nukleotida, key=ucestalost_nukleotida.get)

    broj_nukleotida_dodati = len(duza_sekvenca) - len(kraca_sekvenca)
    dopunjeni_nukleotidi = [najcesci_nukleotid] * broj_nukleotida_dodati
    
    dopunjena_sekvenca = kraca_sekvenca + ''.join(dopunjeni_nukleotidi)
    # print(dopunjena_sekvenca)
        
    return dopunjena_sekvenca, duza_sekvenca

In [23]:
# do ovde je isto kao i hamingovo rastojanje

In [22]:
# 1. pristup
print('p-adicna rastojanja kada krace sekvence dopunjujemo najfrekventnijim kodonom')
for i in range(len(povrsinski_proteini_rnk) - 1):
    for j in range(i+1, len(povrsinski_proteini_rnk)):
        virus1, rnk1 = list(povrsinski_proteini_rnk.items())[i]
        virus2, rnk2 = list(povrsinski_proteini_rnk.items())[j]

        rnk1, rnk2 = izjednacavanje_sekvenci_kodona(rnk1, rnk2)
        
        rnk1_p_brojevi = rnk_sekvenca_u_p_adicne_brojeve(rnk1)
        rnk2_p_brojevi = rnk_sekvenca_u_p_adicne_brojeve(rnk2)

        p_rastojanje = p_adicno_rastojanje(rnk1_p_brojevi, rnk2_p_brojevi)
        print(f'\t{virus1} i {virus2} je {round(p_rastojanje, 5)} Najcesci kodon: {rnk1[-3:]}')


print()

        
 # 2. pristup
print('p-adicna rastojanja kada krace sekvence dopunjujemo najfrekventnijim nukleotidom')
for i in range(len(povrsinski_proteini_rnk) - 1):
    for j in range(i+1, len(povrsinski_proteini_rnk)):
        virus1, rnk1 = list(povrsinski_proteini_rnk.items())[i]
        virus2, rnk2 = list(povrsinski_proteini_rnk.items())[j]

        rnk1, rnk2 = izjednacavanje_sekvenci_nukleotida(rnk1, rnk2)
        
        rnk1_p_brojevi = rnk_sekvenca_u_p_adicne_brojeve(rnk1)
        rnk2_p_brojevi = rnk_sekvenca_u_p_adicne_brojeve(rnk2)

        p_rastojanje = p_adicno_rastojanje(rnk1_p_brojevi, rnk2_p_brojevi)
        print(f'\t{virus1} i {virus2} je {round(p_rastojanje, 5)} Najcesci nukleotid: {rnk1[-1]}')     

p-adicna rastojanja kada krace sekvence dopunjujemo najfrekventnijim kodonom
	mers i bcov je 1095.76 Najcesci kodon: GUU
	mers i sars1 je 1047.88 Najcesci kodon: UUU
	mers i human229e je 1058.64 Najcesci kodon: GUU
	mers i humanoc43 je 1061.76 Najcesci kodon: UAA
	bcov i sars1 je 1042.68 Najcesci kodon: UUU
	bcov i human229e je 1029.32 Najcesci kodon: GUU
	bcov i humanoc43 je 966.44 Najcesci kodon: AAU
	sars1 i human229e je 957.36 Najcesci kodon: GUU
	sars1 i humanoc43 je 1059.76 Najcesci kodon: UUU
	human229e i humanoc43 je 1031.2 Najcesci kodon: GUU

p-adicna rastojanja kada krace sekvence dopunjujemo najfrekventnijim nukleotidom
	mers i bcov je 1095.76 Najcesci nukleotid: U
	mers i sars1 je 1047.88 Najcesci nukleotid: U
	mers i human229e je 1056.96 Najcesci nukleotid: U
	mers i humanoc43 je 1061.76 Najcesci nukleotid: A
	bcov i sars1 je 1042.68 Najcesci nukleotid: U
	bcov i human229e je 1033.92 Najcesci nukleotid: U
	bcov i humanoc43 je 966.44 Najcesci nukleotid: U
	sars1 i human229